**Binary Representation:**
 This is one of the simplest and most widely used representation in
GAs. In this type of representation, the genotype consists of bit
strings.
{1,0,1,1,0}

**Real Valued Representation**
 For problems where we want to define the genes using
continuous rather than discrete variables, the real valued
representation is the most natural.
{0.1, 0.3, 0.1, 0.5, 0.4}

**Integer Representation**
 If we want to encode the four distances – North, South, East and
West, we can encode them as {0,1,2,3}. In such cases, integer
representation is desirable.

**Permutation Representation**
 In many problems, the solution is represented by an order of
elements. In such cases, permutation representation is the most
suited. Example is TSP

In [32]:
import random
import numpy as np

#### CROSSOVERS

In [13]:
def onePointCrossover(x1,x2):
    crossover_point = random.randint(1,len(x1)-1)
    offspring1 =x1[:crossover_point] + x2[crossover_point:]
    offspring2 =x2[:crossover_point] + x1[crossover_point:]
    return offspring1,offspring2

def wholeArithmeticRecombination(x1, x2, alpha=0.5):
    if not (0 <= alpha <= 1):
        raise ValueError("Alpha must be between 0 and 1")

    offspring1 = [alpha * gene1 + (1 - alpha) * gene2 for gene1, gene2 in zip(x1, x2)]
    offspring2 = [(1 - alpha) * gene1 + alpha * gene2 for gene1, gene2 in zip(x1, x2)]

    return offspring1, offspring2

def uniformCrossover(x1, x2, prob=0.5):
    offspring1, offspring2 = [], []

    for gene1, gene2 in zip(x1, x2):
        if random.random() < prob:
            offspring1.append(gene2)
            offspring2.append(gene1)
        else:
            offspring1.append(gene1)
            offspring2.append(gene2)

    return offspring1, offspring2

def multiPointCrossover(x1, x2, num_points):
    if num_points >= len(x1):
        raise ValueError("Number of points must be less than the length of the parents")

    crossover_points = sorted(random.sample(range(1, len(x1)), num_points))
    offspring1, offspring2 = [], []
    toggle = False

    for i in range(len(x1)):
        if i in crossover_points:
            toggle = not toggle
        if toggle:
            offspring1.append(x2[i])
            offspring2.append(x1[i])
        else:
            offspring1.append(x1[i])
            offspring2.append(x2[i])

    return offspring1, offspring2

# Example usage:
x1 = [1, 2, 3, 4, 5, 6]
x2 = [6, 5, 4, 3, 2, 1]
offspring1, offspring2 = multiPointCrossover(x1, x2, 2)
print(offspring1)
print(offspring2)


[1, 2, 3, 4, 2, 6]
[6, 5, 4, 3, 5, 1]


#### MUTATIONS

In [30]:
def bitFlipMutation(chromosome, mutation_rate=0.01):
    mutated_chromosome = chromosome[:]
    for i in range(len(mutated_chromosome)):
        if random.random() < mutation_rate:
            mutated_chromosome[i] = 1 - mutated_chromosome[i]
    return mutated_chromosome

def swapMutation(chromosome):
    mutated_chromosome = chromosome[:]
    idx1, idx2 = random.sample(range(len(chromosome)), 2)
    mutated_chromosome[idx1], mutated_chromosome[idx2] = mutated_chromosome[idx2], mutated_chromosome[idx1]
    return mutated_chromosome

def scrambleMutation(chromosome): # subset chosen and shuffled randomly
    mutated_chromosome = chromosome[:]
    start, end = sorted(random.sample(range(len(chromosome)), 2))
    subset = mutated_chromosome[start:end + 1]
    random.shuffle(subset)
    mutated_chromosome[start:end + 1] = subset
    return mutated_chromosome

def inversionMutation(chromosome): # subset chosen and inverted
    mutated_chromosome = chromosome[:]
    start, end = sorted(random.sample(range(len(chromosome)), 2))
    subset = mutated_chromosome[start:end + 1]
    mutated_chromosome[start:end + 1] = subset[::-1]
    return mutated_chromosome

# Example usage:
chromosome = [0, 1, 0, 1, 1, 0]
mutated = bitFlipMutation(chromosome, mutation_rate=0.5)
print(mutated)


[0, 1, 1, 0, 0, 1]


#### GENETIC ALGO

In [ ]:
#### PARENT SELECTION


# ************Roulette Wheel Selection*********
# 1.Calculate the sum of fitness values
# 2.Generate a random number between 0 and S:
# Traverese through individuals & maintain partialsum(p) if p exceeds S, select that individual as parent


# ************Stochastic Universal Sampling*********(Multi fix points, enabling parent selection in a single spin)
# 1.Calculate the sum of fitness values
# 2.Generate a random number between 0 and S:
# Traverese through individuals & maintain partialsum(p) if p exceeds S, select that individual as parent

# ************Tournament Selection*********
# Each tournament involves selecting a few individuals
# (usually 2 or 3) at random and choosing the best one as a parent.

# Elitism:
# In addition to the selected parents, the
# best individuals from the current
# generation are directly copied to the next
# generation without any changes.



In [33]:
pop_size = 10
min_x =0
max_x =4
max_gen = 100
chromosome_length = 6

items = 'N1 N2 N3 N4 N5 N6'.split()
values = [14, 23, 8, 9, 17, 15]
weights = [1, 3, 7, 4, 5, 6]

print("Items:", items)
print("Values:", values)
print("Weights:", weights)


Items: ['N1', 'N2', 'N3', 'N4', 'N5', 'N6']
Values: [14, 23, 8, 9, 17, 15]
Weights: [1, 3, 7, 4, 5, 6]


In [36]:
def create_chromosome():
    return ''.join(random.choice(['0','1']) for _ in range(chromosome_length))

# Initialize the population
population=[create_chromosome() for _ in range (pop_size)]
population

['110110',
 '110110',
 '001001',
 '110011',
 '101100',
 '110110',
 '011101',
 '011111',
 '001110',
 '110001']

In [37]:
def fitness_func(population, values, weights):
    fitnessList = []
    for chromosome in population:
        val = 0
        wgt = 0
        index = 0
        while index < len(chromosome):
            if int(chromosome[index]) == 1:
                val += values[index]
                wgt += weights[index]
            index+=1
        if wgt > 10:
            fitnessList.append(0)
        else:
            fitnessList.append(val)
            
    return fitnessList
#define the mutation function(random perturbation)
mut_prob = 0.1
def mutation(x):
    x_list = list(map(int, x))
    for i in range(len(x_list)):
        if random.uniform(0, 1) < mut_prob:
            if x_list[i] == 0:
                x_list[i] = 1
            else:
                x_list[i] = 0
    x_str = ''.join(map(str, x_list))
    return x_str



def crossover(x1,x2):
    crossover_point = random.randint(1,len(x1)-1)
    offspring1 =x1[:crossover_point] + x2[crossover_point:]
    offspring2 =x2[:crossover_point] + x1[crossover_point:]
    return offspring1,offspring2

def mainn():
    for i in range(max_gen):
        fitness_scoeres = fitness_func(population, values, weights)

        selected_chromosomes = random.sample(population, 2)
        print("Selected Chromosomes for Crossover:")
        print(selected_chromosomes[0])
        print(selected_chromosomes[1])
        offspring1, offspring2 = crossover(selected_chromosomes[0], selected_chromosomes[1])
        print("Offsprings after Crossover:")
        print(offspring1)
        print(offspring2)

        min_index = fitness_scoeres.index(min(fitness_scoeres))
        if random.randint(1,2) == 1:
            offspring1 = mutation(offspring1)
            population[min_index] = offspring1
        else:
            offspring2 = mutation(offspring2)
            population[min_index] = offspring2
        
        score = fitness_func(population, values, weights)
        print(population[score.index(max(fitness_scoeres))], max(fitness_scoeres))

mainn()

Selected Chromosomes for Crossover:
110011
011111
Offsprings after Crossover:
110011
011111
110001 52
Selected Chromosomes for Crossover:
001110
011101
Offsprings after Crossover:
001101
011110
110001 52
Selected Chromosomes for Crossover:
011111
001110
Offsprings after Crossover:
011110
001111
110001 52
Selected Chromosomes for Crossover:
110001
001110
Offsprings after Crossover:
110010
001101
110001 52
Selected Chromosomes for Crossover:
110010
011101
Offsprings after Crossover:
110011
011100
110010 54
Selected Chromosomes for Crossover:
101100
110001
Offsprings after Crossover:
101101
110000
110010 54
Selected Chromosomes for Crossover:
001001
011111
Offsprings after Crossover:
001001
011111
110010 54
Selected Chromosomes for Crossover:
011111
001001
Offsprings after Crossover:
011001
001111
110010 54
Selected Chromosomes for Crossover:
110001
110010
Offsprings after Crossover:
110010
110001
110010 54
Selected Chromosomes for Crossover:
110010
101100
Offsprings after Crossover:
1111